In [1]:
#导入jieba
import jieba

#导入停用词表
with open('stopwords_total.txt','r',encoding="utf-8") as scu_stopwords:
    stopwords = set(scu_stopwords.read().splitlines()) 

#词典
#jieba.load_userdict(词典地址)
#因为在导出的时候可以选择编码，所以这里导入的txt文件直接就是utf-8编码
# 读取文件
with open("txt_general_cut.txt", "r", encoding="utf-8") as txt:
    content = txt.read()

# 分词

In [ ]:
# 分词
words1 = jieba.lcut(content)
# 去除停用词，同时除去字数<2的词
words2 = [w for w in words1 if w not in stopwords]

# 写入分词结果
with open("stage1cut2.txt", "w", encoding="utf-8") as result:
    result.write(" ".join(words2))

## 分词结果处理

In [42]:
# 词频统计2
counts2 = {}
for word in words2:
    if len(word) == 1:
        continue
    counts2[word] = counts2.get(word, 0) + 1

# 排序
items2 = sorted(counts2.items(), key=lambda x: x[1], reverse=True)

# 输出和写入高频词
with open("stage1highFrequency2.txt", "a", encoding="utf-8") as result:
    for i in range(min(40, len(items2))):
        word, count = items2[i]
        print(u"{0:<10}{1:>5}".format(word, count))
        result.write(f"{word}: {count}\n")

病人          190
医生          146
手术           95
工作           78
露西           72
生命           70
死亡           65
神经外科         60
时间           58
生活           55
孩子           54
癌症           53
治疗           46
肿瘤           45
住院医生         43
意义           42
医院           41
希望           40
艾玛           38
事情           34
手术室          32
护士           29
病房           29
研究           28
尸体           28
大脑           28
科学           28
发现           27
未来           27
痛苦           27
接受           26
医学           26
人生           26
告诉           26
主治医生         26
面对           26
医学院          26
神经           26
世界           25
身体           24


In [43]:
##检查是否为utf=8

In [44]:
import chardet

with open('stage1cut1.txt', 'rb') as f:
    data = f.read()
    result = chardet.detect(data)
    print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


## LDA

In [45]:
train2 = []

with open("stage1cut2.txt", "r", encoding="utf-8") as wordspilt:
    for line in wordspilt:
        if line != '':
            line = line.split()
            train2.append([w for w in line])
            #这里每一个元素是一个词列表,代表一篇文档(此处是代表一个句子)
            #后期看看这里能不能优化？还是其实不管分成几个文档，结果都一样？

In [61]:
from gensim import corpora
from gensim.models import LdaModel

dictionary2 = corpora.Dictionary(train2)
corpus2 = [dictionary2.doc2bow(text) for text in train2]

#训练LDA
#num_topics=5，即输出5个主题
lda2 = LdaModel(corpus=corpus2, id2word=dictionary2, num_topics=3, passes=60)

In [62]:
#输出每个主题的关键词和权重
#num_words=10，即输出10个词/主题
for topic in lda2.print_topics(num_words=50):
    termNumber = topic[0]
    print(topic[0], ':', sep='')
    listOfTerms = topic[1].split('+')
    for term in listOfTerms:
        listItems = term.split('*')
        print(' ', listItems[1], '(', listItems[0], ')', sep='')


0:
 "说" (0.014)
 "做" ( 0.008)
 "手术" ( 0.007)
 "病人" ( 0.006)
 "时间" ( 0.005)
 "想" ( 0.004)
 "死亡" ( 0.004)
 "医生" ( 0.003)
 "生命" ( 0.003)
 "里" ( 0.003)
 "住院医生" ( 0.003)
 "中" ( 0.003)
 "…" ( 0.003)
 "大脑" ( 0.003)
 "工作" ( 0.003)
 "孩子" ( 0.002)
 "医院" ( 0.002)
 "医学院" ( 0.002)
 "治疗" ( 0.002)
 "露西" ( 0.002)
 "更" ( 0.002)
 "时" ( 0.002)
 "完" ( 0.002)
 "手术室" ( 0.002)
 "生活" ( 0.002)
 "面对" ( 0.002)
 "走" ( 0.002)
 "喜欢" ( 0.002)
 "神经" ( 0.002)
 "缝合" ( 0.002)
 "化疗" ( 0.002)
 "慢慢" ( 0.002)
 "过程" ( 0.002)
 "研究" ( 0.002)
 "明白" ( 0.002)
 "告诉" ( 0.001)
 "主治医生" ( 0.001)
 "意义" ( 0.001)
 "希望" ( 0.001)
 "神经外科" ( 0.001)
 "实验室" ( 0.001)
 "吃" ( 0.001)
 "前" ( 0.001)
 "找" ( 0.001)
 "问" ( 0.001)
 "选择" ( 0.001)
 "数据" ( 0.001)
 "躺" ( 0.001)
 "结束" ( 0.001)
 "V"( 0.001)
1:
 "…" (0.008)
 "医生" ( 0.006)
 "病人" ( 0.005)
 "说" ( 0.005)
 "做" ( 0.004)
 "神经外科" ( 0.004)
 "中" ( 0.003)
 "没" ( 0.003)
 "死亡" ( 0.003)
 "癌症" ( 0.003)
 "工作" ( 0.003)
 "更" ( 0.003)
 "里" ( 0.002)
 "时" ( 0.002)
 "手术" ( 0.002)
 "生活" ( 0.002)
 "想" ( 0.002)
 "治疗" 

# 可视化

## 检查

In [63]:
# 检查 corpus 是否含复数
for doc in corpus2:
    for id, freq in doc:
        if isinstance(freq, complex):
            print(f"复数出现: {id}, {freq}")

# 检查 dictionary 是否正常
#print(dictionary2.token2id)

## 进行可视化

In [64]:
import pyLDAvis
import pyLDAvis.gensim

### LDA2
graph2 = pyLDAvis.gensim.prepare(lda2,corpus2,dictionary2)
pyLDAvis.save_html(graph2,"lda_vis_total.html")

In [60]:
# pyLDAvis.show(graph)